In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model

# Load your cleaned dataset (replace 'cleaned_dataset.csv' with the actual file name)
df_cleaned = pd.read_csv('cleaned_dataset.csv')

# Assuming 'target' is the column you want to predict
target_column = 'Prediction'

# Split the dataset into features (X) and target variable (y)
X = df_cleaned.drop(columns=[target_column])
y = df_cleaned[target_column]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data (important for neural networks)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize the Neural Network model
model = Sequential()

# Add input layer and first hidden layer
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))

# Add output layer
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model on the training set
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Make predictions on the test set
y_pred_prob = model.predict(X_test)
y_pred = [1 if prob >= 0.5 else 0 for prob in y_pred_prob]

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)


Epoch 1/10
104/104 [==============================] - 1s 5ms/step - loss: 0.2674 - accuracy: 0.8982 - val_loss: 0.1366 - val_accuracy: 0.9589
Epoch 2/10
104/104 [==============================] - 0s 4ms/step - loss: 0.0811 - accuracy: 0.9779 - val_loss: 0.1300 - val_accuracy: 0.9396
Epoch 3/10
104/104 [==============================] - 0s 4ms/step - loss: 0.0520 - accuracy: 0.9882 - val_loss: 0.0943 - val_accuracy: 0.9771
Epoch 4/10
104/104 [==============================] - 0s 4ms/step - loss: 0.0409 - accuracy: 0.9873 - val_loss: 0.0974 - val_accuracy: 0.9674
Epoch 5/10
104/104 [==============================] - 0s 4ms/step - loss: 0.0275 - accuracy: 0.9949 - val_loss: 0.0868 - val_accuracy: 0.9698
Epoch 6/10
104/104 [==============================] - 0s 4ms/step - loss: 0.0237 - accuracy: 0.9952 - val_loss: 0.0868 - val_accuracy: 0.9686
Epoch 7/10
104/104 [==============================] - 0s 4ms/step - loss: 0.0199 - accuracy: 0.9973 - val_loss: 0.1006 - val_accuracy: 0.9638
Epoch 